In [1]:
from pathlib import Path
import sys

root_dir = Path().resolve().parent.parent.parent.as_posix()
if root_dir not in sys.path:
    sys.path.append(root_dir)
del root_dir

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import efficientnet_v2
import tensorflow as tf

from utils import config, datasets, models

In [2]:
MODEL_NAME = 'efficientnetv2s-5_layer_dense-he_normal-rescaling'
EPOCHS = 200

In [3]:
train_dataset, validation_dataset, test_dataset = datasets.load_dataset()

Found 6400 files belonging to 4 classes.


In [4]:
data_augmentation = keras.Sequential([
    layers.Rescaling(1. / 255),
    layers.RandomFlip(),
    layers.RandomRotation(factor=0.25)
])

In [5]:
top = keras.Sequential([
    layers.Dense(2048, activation=keras.activations.relu, kernel_initializer="he_normal"),
    layers.Dropout(0.2),
    layers.Dense(2048, activation=keras.activations.relu, kernel_initializer="he_normal"),
    layers.Dropout(0.2),
    layers.Dense(2048, activation=keras.activations.relu, kernel_initializer="he_normal"),
    layers.Dropout(0.2),
    layers.Dense(2048, activation=keras.activations.relu, kernel_initializer="he_normal"),
    layers.Dense(4, activation=keras.activations.softmax, kernel_initializer="he_normal")
])

In [6]:
model = models.create_model(
    model_name=MODEL_NAME,
    preprocessing_layers=data_augmentation,
    base_model=efficientnet_v2.EfficientNetV2S,
    top_layers=top,
    pooling='avg',
)

In [7]:
history = models.fit_model(
    model,
    train_data=train_dataset,
    validation_data=validation_dataset,
    epochs=EPOCHS
)

Epoch 1/200
160/160 [==============================] - 44s 169ms/step - loss: 3.1464 - sparse_categorical_accuracy: 0.4648 - val_loss: 1.0117 - val_sparse_categorical_accuracy: 0.5172
Epoch 2/200
160/160 [==============================] - 23s 145ms/step - loss: 1.1293 - sparse_categorical_accuracy: 0.4721 - val_loss: 1.0161 - val_sparse_categorical_accuracy: 0.5063
Epoch 3/200
160/160 [==============================] - 23s 145ms/step - loss: 1.0189 - sparse_categorical_accuracy: 0.4859 - val_loss: 0.9931 - val_sparse_categorical_accuracy: 0.5172
Epoch 4/200
160/160 [==============================] - 23s 145ms/step - loss: 1.0255 - sparse_categorical_accuracy: 0.4945 - val_loss: 1.0162 - val_sparse_categorical_accuracy: 0.4219
Epoch 5/200
160/160 [==============================] - 23s 145ms/step - loss: 1.0424 - sparse_categorical_accuracy: 0.4949 - val_loss: 1.0048 - val_sparse_categorical_accuracy: 0.5172
Epoch 6/200
160/160 [==============================] - 23s 145ms/step - loss: 1.

In [8]:
model.summary()

Model: "efficientnetv2s-5_layer_dense-he_normal-rescaling"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 128, 128, 3)       0         
                                                                 
 efficientnetv2-s (Functiona  (None, 1280)             20331360  
 l)                                                              
                                                                 
 sequential_1 (Sequential)   (None, 4)                 15220740  
                                                                 
Total params: 35,552,100
Trainable params: 15,220,740
Non-trainable params: 20,331,360
_________________________________________________________________
